In [3]:
# Retrieval time ratio

import os
import sys
import logging
import argparse
import itertools

import numpy as np
from joblib import Parallel, delayed

sys.path.insert(0, '../../../network')
from network import Population, RateNetwork
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse, ThresholdPlasticityRule
from sequences import GaussianSequence

logging.basicConfig(level=logging.INFO)

In [4]:
cpu_cores = 4
n_realizations = 1

In [5]:
def simulate_network(S, T, theta, sigma):

    phi = ErrorFunction(mu=theta, sigma=sigma).phi
    exc = Population(N=40000, tau=1e-2, phi=phi)

    sequences = [GaussianSequence(S,exc.size,seed=0)]
    patterns = np.stack([s.inputs for s in sequences])

    conn_EE = SparseConnectivity(source=exc, target=exc, p=0.005)
    synapse = LinearSynapse(conn_EE.K, A=1)
    conn_EE.store_sequences(patterns, synapse.h_EE)

    net = RateNetwork(exc, c_EE=conn_EE, formulation=1)

    r0 = exc.phi(patterns[0,0,:])
    net.simulate(T, r0=r0)

    overlaps = sequences[0].overlaps(net, exc)
    t_max = overlaps[-1,:].argmax()

    return t_max

In [ ]:
S = [8,16,32,64]
T = [0.15,0.3,0.6,1.0]
theta = np.linspace(0,0.2,10)
sigma = np.linspace(0.01,0.25,10)
theta_0, sigma_0 = 0.2, 0.15

def func_a(i,j):
    t_max = simulate_network(S[i], T[i], theta[j], sigma_0)
    return t_max

def func_b(i,j):
    t_max = simulate_network(S[i], T[i], theta_0, sigma[j])
    return t_max

# Fixed sigma, varying theta
t_max_theta = Parallel(n_jobs=cpu_cores)(
        delayed(func_a)(i,j) for i,j in itertools.product(range(len(S)),range(len(theta))))

# Fixed theta, varying sigma
t_max_sigma = Parallel(n_jobs=cpu_cores)(
        delayed(func_b)(i,j) for i,j in itertools.product(range(len(S)),range(len(sigma))))

In [ ]:
logging.info("Saving data")
np.save(open("data/data.npy", "wb"), [S, theta, sigma, t_max_theta, t_max_sigma])